# Searching for the category

## We are going to categorize products 

In [1]:
import pandas as pd


In [2]:
# product_category_df.csv
#url = "https://drive.google.com/file/d/1jzEITeNiUGIMiljfp7XnDsh-U2Hyr3M-/view?usp=sharing"
#path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
#product_category_df = pd.read_csv(path)

# products.csv (local)
products_qu = pd.read_csv("../data/quality-assessed-data/products_qu.csv")
brands = pd.read_csv("../data/raw-data/brands.csv", dtype=str)


In [3]:
product_category_df = products_qu.copy()


# This 10-category system groups our 125 product types into meaningful business categories:
1. **Cases & Protection** - All protective accessories (iPhone cases, iPad cases, MacBook cases, screen protectors, etc.)
2. **Computers & Laptops** - Computing devices (MacBooks, iMacs, refurbished computers)
3. **Storage** - All storage solutions (external drives, internal drives, SSDs, RAM, USB storage, NAS)
4. **Cables & Adapters** - Connectivity solutions (cables, adapters, docks, hubs)
5. **Power & Charging** - Power solutions (chargers, power banks, charging stations)
6. **Audio & Input** - Audio devices and input accessories (headphones, speakers, keyboards, mice, styluses)
7. **Mobile Accessories** - Mobile device accessories (iPhone/iPad accessories, Apple Watch, stands, cable management)
8. **Displays & Monitors** - Display solutions (monitors, displays)
9. **Professional Tools** - Creative and professional equipment (graphics tablets, Apple TV, PCIe cards, surveillance)
10. **Services** - All repair and service offerings (iPhone repair, MacBook repair, various service tiers)

This approach:
- **Eliminates "Other"** by using smart fallback categorization
- **Combines similar products** (e.g., all case types go to "Cases & Protection")
- **Creates business-meaningful categories** for analysis and reporting
- **Covers all 125 types** with logical groupings
- **Maintains analytical value** while simplifying the category structure


In [4]:
# Normalize the "type" column into a consistent, comparable string
def normalize_type(raw):
    s = str(raw).strip()
    if not s or s.lower() in {"nan", "none"}:
        return ""
    # Unify decimal separator and case
    s = s.replace(",", ".").upper()

    # Keep known scientific-notation tokens as-is (avoid float reformatting)
    known_sci = {
        "5.49E+11",  # battery cases
        "1.44E+11", "1.47E+11", "1.49E+11", "1.51E+11", "1.53E+11", "1.55E+11", "1.57E+11",
        "1.59E+11", "1.61E+11", "1.63E+11", "1.65E+11", "1.67E+11", "1.69E+11",
        "1.68E+11", "1.70E+11", "1.72E+11", "1.74E+11", "1.76E+11", "1.78E+11", "1.80E+11",
        "1.82E+11", "1.84E+11", "1.85E+11", "1.87E+11", "1.89E+11", "1.91E+11", "1.93E+11",
        "1.95E+11", "1.97E+11", "1.99E+11", "2.01E+11", "2.03E+11",
        "5.74E+15",  # iMac family
    }
    if s in known_sci:
        return s

    # If it's a pure integer-like value (possibly with .0), reduce to int string
    if re.fullmatch(r"\d+(\.0+)?", s):
        try:
            return str(int(float(s)))
        except:
            return s

    return s

product_category_df["type"] = product_category_df["type"].map(normalize_type)

# Direct mapping for the most common/clear codes
type_to_category = {
    # Cables & Adapters
    "1325": "Cables & Adapters",
    "12585395": "Cables & Adapters",
    "1334": "Cables & Adapters",
    "13955395": "Cables & Adapters",
    "1230": "Cables & Adapters",
    "12995397": "Cables & Adapters",

    # Storage
    "12175397": "Storage",     # NAS & bundles
    "11935397": "Storage",     # External storage/enclosures
    "12655397": "Storage",     # Internal HDDs
    "57445397": "Storage",     # USB flash/memory cards
    "12215397": "Storage",     # SSD families (e.g., Aura/Pro/EVO)
    "12755395": "Storage",     # Optical bay / adapters
    "1433": "Storage",         # SSD upgrade kits
    "1364": "Storage",         # Memory/RAM

    # Power & Charging
    "13005399": "Power & Charging",
    "13615399": "Power & Charging",
    "1515": "Power & Charging",

    # Audio & Input
    "5384": "Audio & Input",   # Headphones / headsets
    "5398": "Audio & Input",   # Bluetooth speakers / car kits
    "13855401": "Audio & Input", # Keyboards / keypads
    "1387": "Audio & Input",     # Mice
    "1229": "Audio & Input",     # Stylus / pens

    # Mobile & Accessories
    "8696": "Mobile & Accessories",     # Stands/supports
    "1216": "Mobile & Accessories",
    "12285400": "Mobile & Accessories",
    "12355400": "Mobile & Accessories",

    # Cases & Protection (product-line specific codes; more via suffix rule below)
    "11865403": "Cases & Protection",  # iPhone cases/covers
    "12635403": "Cases & Protection",  # iPad cases/anti-theft mounts
    "13835403": "Cases & Protection",  # MacBook cases/sleeves
    "13555403": "Cases & Protection",  # iPhone screen protectors
    "14035403": "Cases & Protection",  # iPad screen protectors

    # Displays & Monitors
    "1296": "Displays & Monitors",

    # Professional Tools
    "1405": "Professional Tools",     # Graphics tablets
    "101781405": "Professional Tools",
    "1276": "Professional Tools",     # PCIe/expansion cards
    "4259": "Professional Tools",     # Apple TV/media players
    "1404": "Professional Tools",     # NAS/IP camera accessories

    # Computers & Laptops
    "1282": "Computers & Laptops",    # MacBook Pro & variants
    "2158": "Computers & Laptops",
    "1298": "Computers & Laptops",    # Open-box/refurb families
    "5.74E+15": "Computers & Laptops",# iMac retina families

    # Scientific-notation specials
    "5.49E+11": "Cases & Protection", # iPhone battery cases
}
service_codes = {
    # Repairs/services families (scientific-notation tokens)
    "1.44E+11", "1.47E+11", "1.49E+11", "1.51E+11", "1.53E+11", "1.55E+11", "1.57E+11",
    "1.59E+11", "1.61E+11", "1.63E+11", "1.65E+11", "1.67E+11", "1.69E+11",
    "1.68E+11", "1.70E+11", "1.72E+11", "1.74E+11", "1.76E+11", "1.78E+11", "1.80E+11",
    "1.82E+11", "1.84E+11", "1.85E+11", "1.87E+11", "1.89E+11", "1.91E+11", "1.93E+11",
    "1.95E+11", "1.97E+11", "1.99E+11", "2.01E+11", "2.03E+11",
}

# Suffix-based fallbacks for unseen codes (robust to new values)
def infer_category_from_suffix(t):
    if not t:
        return ""
    # Common department suffixes
    if t.endswith("403"):
        return "Cases & Protection"
    if t.endswith("397"):
        return "Storage"
    if t.endswith("399"):
        return "Power & Charging"
    if t.endswith("395"):
        return "Cables & Adapters"
    if t.endswith("400"):
        return "Mobile & Accessories"
    return ""

# Build the category column
def categorize(t):
    # direct map
    if t in type_to_category:
        return type_to_category[t]
    # services
    if t in service_codes:
        return "Services"
    # suffix inference
    fallback = infer_category_from_suffix(t)
    if fallback:
        return fallback
    return "Other"

product_category_df["category"] = product_category_df["type"].map(categorize)

# (optional) Quick sanity check
#print(product_category_df["category"].value_counts(dropna=False).head(10))
#product_category_df

NameError: name 're' is not defined

In [5]:
def infer_from_text(name, desc):
    text = f"{str(name)} {str(desc)}".lower()

    # Cases & Protection
    if re.search(r"\b(case|cover|sleeve|shell|protector|bookbook|folio|bumper|tempered glass)\b", text):
        return "Cases & Protection"

    # Storage
    if re.search(r"\b(ssd|hdd|hard ?drive|disk|nas|raid|enclosure|dock sata|backup plus|my cloud|rugged)\b", text):
        return "Storage"

    # Cables & Adapters
    if re.search(r"\b(adapter|cable|thunderbolt|displayport|mini displayport|hdmi|vga|dvi|usb[- ]c?|ethernet)\b", text):
        return "Cables & Adapters"

    # Power & Charging
    if re.search(r"\b(charger|magsafe|power ?bank|battery(?! case)|powerstation|charging dock|wireless charge)\b", text):
        return "Power & Charging"

    # Audio & Input (headphones, speakers, mice, keyboards, stylus)
    if re.search(r"\b(headphone|earpods?|earphones?|headset|speaker|mouse|mice|keyboard|keypad|stylus|pen)\b", text):
        return "Audio & Input"

    # Mobile & Accessories (stands, docks, holders, tripods, mounts)
    if re.search(r"\b(stand|dock(?!ing station)|holder|mount|tripod|bike ?mount|numpad|cable ?management)\b", text):
        return "Mobile & Accessories"

    # Displays & Monitors
    if re.search(r"\b(monitor|display|retina display|uhd|4k|qhd|27[\"”]|21\.5[\"”]|24[\"”])\b", text):
        return "Displays & Monitors"

    # Professional Tools (graphics tablets, PCIe, IP camera kits)
    if re.search(r"\b(wacom|intuos|cintiq|graphics tablet|pcie|ip camera|rackstation|synology|qnap)\b", text):
        return "Professional Tools"

    # Services
    if re.search(r"\b(applecare|repair|service|warranty)\b", text):
        return "Services"

    return None

mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
inferred = product_category_df.loc[mask_other].apply(
    lambda r: infer_from_text(r.get("name", ""), r.get("desc", "")),
    axis=1
)

product_category_df.loc[mask_other & inferred.notna(), "category"] = inferred[ inferred.notna() ]
product_category_df["category"] = product_category_df["category"].fillna("Other")

# Optional: see how many null-type rows were upgraded
null_type_mask = product_category_df["type"].isna() | (product_category_df["type"].astype(str).str.strip()=="")
improved = (null_type_mask & product_category_df["category"].ne("Other")).sum()
total_null_type = null_type_mask.sum()
print(f"Inferred categories for {improved} of {total_null_type} rows that had null/blank type.")


Inferred categories for 7 of 46 rows that had null/blank type.


In [6]:

# Expand direct mapping for codes observed as Other
extra_type_to_category = {
    # Services / warranties
    "1231": "Services",

    # Power & Charging
    "5395": "Power & Charging",
    "24215399": "Power & Charging",

    # Mobile & Accessories (consumer mounts/stands)
    "5720": "Mobile & Accessories",

    # Cases & Protection (bags/backpacks, sleeves)
    "1392": "Cases & Protection",

    # Phones and similar device families (map to Mobile & Accessories bucket)
    "51601716": "Mobile & Accessories",

    # Smart cams / consumer IoT accessories (treat as Mobile & Accessories)
    "9094": "Mobile & Accessories",

    # Laptop batteries (internal replacements)
    "10142": "Power & Charging",
}

# Apply the extended map to remaining Unknowns
mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
product_category_df.loc[mask_other, "category"] = product_category_df.loc[mask_other, "type"].map(
    extra_type_to_category
).fillna(product_category_df.loc[mask_other, "category"])


# Stronger, multilingual keyword inference from name/desc
def infer_from_text_strong(name, desc):
    text = f"{str(name)} {str(desc)}".lower()

    # Cases & Protection
    if re.search(r"\b(case|cover|sleeve|shell|protector|bumper|folio|tempered glass|glass templado|bookbook|mochila|backpack|bag|malet[ií]n|carcasa|funda|estuche)\b", text):
        # Battery case heuristic -> still Cases & Protection
        return "Cases & Protection"

    # Storage
    if re.search(r"\b(ssd|hdd|hard ?drive|disk|disco|unidad|pendrive|flash ?drive|usb 3\.0|usb3|tarjeta|microsd|sdxc|sdhc|nas|raid|enclosure|carcasa.*(disco|hdd|ssd)|dock sata|backup plus|my cloud|rugged)\b", text):
        return "Storage"

    # Cables & Adapters
    if re.search(r"\b(adapter|adaptador|cable|convert(ido|e)r|hub|dock(?!ing station)|thunderbolt|display ?port|mini ?display ?port|mdp|hdmi|vga|dvi|usb[- ]?c|usb-c|ethernet|gigabit|multiport)\b", text):
        return "Cables & Adapters"

    # Power & Charging
    if re.search(r"\b(charger|cargador|magsafe|power ?bank|powerbank|battery pack|wireless charg|charging (dock|stand)|carga( r[aá]pida)?|cargador coche)\b", text):
        return "Power & Charging"

    # Audio & Input
    if re.search(r"\b(headphone|earpods?|earphones?|headset|auriculares|cascos|altavoz|speaker|micr[oó]fono|mouse|rat[oó]n|keyboard|teclado|keypad|stylus|pen|l[aá]piz)\b", text):
        return "Audio & Input"

    # Mobile & Accessories
    if re.search(r"\b(stand|soporte|holder|mount|tr[ií]pode|bike ?mount|watch ?band|correa|numpad|cable ?management|airpods|car mount|car holder)\b", text):
        return "Mobile & Accessories"

    # Displays & Monitors
    if re.search(r"\b(monitor|display|retina display|uhd|4k|qhd|wqhd|ips|27[\"”]|21\.5[\"”]|24[\"”])\b", text):
        return "Displays & Monitors"

    # Professional Tools
    if re.search(r"\b(wacom|intuos|cintiq|graphics tablet|tableta gr[aá]fica|pcie|pci[- ]e|ip camera|c[aá]mara ip|rackstation|synology|qnap|nas)\b", text):
        return "Professional Tools"

    # Services
    if re.search(r"\b(applecare|repair|service|warranty|garant[ií]a|reparaci[oó]n)\b", text):
        return "Services"

    return None

mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
inferred2 = product_category_df.loc[mask_other].apply(
    lambda r: infer_from_text_strong(r.get("name", ""), r.get("desc", "")),
    axis=1
)
product_category_df.loc[mask_other & inferred2.notna(), "category"] = inferred2[inferred2.notna()]


# Brand/SKU-prefix fallback for stubborn cases
def brand_default_from_sku(sku):
    code = str(sku).strip().upper()[:3]
    return {
        # Storage-oriented brands
        "LAC": "Storage", "WDT": "Storage", "SEA": "Storage", "SAN": "Storage",
        "OWC": "Storage", "CRU": "Storage", "KIN": "Storage", "LEX": "Storage",

        # Displays
        "BNQ": "Displays & Monitors", "LGE": "Displays & Monitors",

        # Cables & Adapters
        "MOS": "Cables & Adapters", "SAT": "Cables & Adapters", "BEL": "Cables & Adapters",
        "KEN": "Cables & Adapters", "NTE": "Cables & Adapters", "HGD": "Cables & Adapters",

        # Power & Charging defaults (general power brands/items)
        "MOP": "Power & Charging", "BEL": "Power & Charging",  # BEL will be mostly corrected by text earlier

        # Professional Tools
        "WAC": "Professional Tools", "SNN": "Professional Tools", "SYN": "Professional Tools", "QNA": "Professional Tools",

        # Mobile & Accessories / stands
        "ELA": "Mobile & Accessories", "JMO": "Mobile & Accessories", "IST": "Mobile & Accessories",
        "IOT": "Mobile & Accessories", "IKM": "Mobile & Accessories", "TWS": "Mobile & Accessories",
        "MUV": "Mobile & Accessories", "GRT": "Mobile & Accessories", "HGD": "Mobile & Accessories",
        "STM": "Cases & Protection",  # STM is mostly cases
    }.get(code)

mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
brand_fallback = product_category_df.loc[mask_other, "sku"].map(brand_default_from_sku)
product_category_df.loc[mask_other & brand_fallback.notna(), "category"] = brand_fallback[brand_fallback.notna()]

# Final safety fill
product_category_df["category"] = product_category_df["category"].fillna("Other")

# Report remaining Unknowns
remaining_other = (product_category_df["category"] == "Other").sum()
print(f"Remaining Other after expanded rules: {remaining_other}")

Remaining Other after expanded rules: 1063


In [7]:

import re
import pandas as pd

# Ensure we have a 'brand' column on product_category_df (from SKU prefix) and a clean text corpus
if "brand" not in product_category_df.columns or product_category_df["brand"].isna().all():
    try:
        brands = pd.read_csv("../data/raw-data/brands.csv", dtype=str)
        brands.columns = brands.columns.str.strip().str.lower()  # expect columns: short,long
        brands["short"] = brands["short"].str.strip().str.upper()
        brands["long"]  = brands["long"].str.strip()
        code_to_brand = dict(zip(brands["short"], brands["long"]))
        product_category_df["sku"] = product_category_df["sku"].astype(str).str.strip().str.upper()
        product_category_df["brand"] = product_category_df["sku"].str[:3].map(code_to_brand)
    except Exception:
        product_category_df["brand"] = None  # fallback if brands.csv not available

# If still missing, try to extract brand from the beginning of the product name
def guess_brand_from_name(name):
    if not name or not isinstance(name, str):
        return None
    head = name.strip().split()[0]
    # Normalize common punctuation
    head = re.sub(r"[^A-Za-z0-9\-\.\+]", "", head)
    # Known top brands in this dataset domain
    known = {
        "APPLE","LACIE","SANDISK","SEAGATE","WD","WESTERN","MOPHIE","BELKIN","MOSHI","SATECHI","WACOM",
        "KINGSTON","CRUCIAL","OWC","SYNOLOGY","QNAP","BENQ","LG","KEN","KENSINGTON","LMP","JUST","STM",
        "GRIFFIN","TADO","PHILIPS","NEWERTECH","ITO","IOTTIE","IK","IKMULTIMEDIA","WITHINGS","BEEZ","BE.EZ",
        "SPECK","SENNHEISER","ELAGO","HENGE","MACLOCKS","BLA","BLUEL"
    }
    up = head.upper()
    if up in known:
        return head.title()
    # Handle e.g., "WD" => Western Digital
    if up == "WD":
        return "Western Digital"
    if up == "APPLE":
        return "Apple"
    return None

missing_brand_mask = product_category_df["brand"].isna() | (product_category_df["brand"].astype(str).str.strip() == "")
product_category_df.loc[missing_brand_mask, "brand"] = product_category_df.loc[missing_brand_mask, "name"].apply(guess_brand_from_name)
product_category_df["brand"] = product_category_df["brand"].fillna("Other/Other")

# Text-based inference rules (multilingual, broad coverage), independent of type
def infer_category_from_text_strong(name, desc):
    text = f"{str(name)} {str(desc)}".lower()

    # Cases & Protection
    if re.search(r"\b(case|cover|sleeve|shell|protector|bumper|folio|tempered\s*glass|bookbook|mochila|backpack|bag|malet[ií]n|carcasa|funda|estuche)\b", text):
        return "Cases & Protection"

    # Storage
    if re.search(r"\b(ssd|hdd|hard\s?drive|disk|disco|unidad|pendrive|flash\s?drive|usb\s?3(\.0)?\b|usb3\b|tarjeta|microsd|sdxc|sdhc|nas|raid|enclosure|carcasa.*(disco|hdd|ssd)|dock\s*sata|backup\s*plus|my\s*cloud|rugged)\b", text):
        return "Storage"

    # Cables & Adapters
    if re.search(r"\b(adapter|adaptador|cable|convert(ido|e)r|hub|dock(?!ing\s*station)|thunderbolt|display\s?port|mini\s?display\s?port|mdp|hdmi|vga|dvi|usb[-\s]?c|ethernet|gigabit|multiport)\b", text):
        return "Cables & Adapters"

    # Power & Charging
    if re.search(r"\b(charger|cargador|magsafe|power\s?bank|powerbank|battery\s?pack|wireless\s?charg|charging\s*(dock|stand)|carga(\s*r[aá]pida)?|cargador\s*coche|powerstation)\b", text):
        return "Power & Charging"

    # Audio & Input
    if re.search(r"\b(headphone|earpods?|earphones?|headset|auriculares|cascos|altavoz|speaker|micr[oó]fono|microphone|mouse|rat[oó]n|keyboard|teclado|keypad|stylus|pen|l[aá]piz)\b", text):
        return "Audio & Input"

    # Mobile & Accessories
    if re.search(r"\b(stand|soporte|holder|mount|tr[ií]pode|tripod|bike\s*mount|watch\s*band|correa|numpad|cable\s*management|airpods|car\s*(mount|holder)|dock\s*stand)\b", text):
        return "Mobile & Accessories"

    # Displays & Monitors
    if re.search(r"\b(monitor|display|retina\s*display|uhd|4k|qhd|wqhd|ips|27[\"”]|21\.5[\"”]|24[\"”])\b", text):
        return "Displays & Monitors"

    # Professional Tools
    if re.search(r"\b(wacom|intuos|cintiq|graphics\s*tablet|tableta\s*gr[aá]fica|pcie|pci[-\s]?e|ip\s*camera|c[aá]mara\s*ip|rackstation|synology|qnap|nas)\b", text):
        return "Professional Tools"

    # Services
    if re.search(r"\b(applecare|repair|service|warranty|garant[ií]a|reparaci[oó]n|soporte\s*t[eé]cnico)\b", text):
        return "Services"

    return None

# Brand-driven defaults (only used when text rules don’t decide)
brand_defaults = {
    # Storage-oriented brands
    "LaCie": "Storage", "Western Digital": "Storage", "Seagate": "Storage", "SanDisk": "Storage",
    "OWC": "Storage", "Crucial": "Storage", "Kingston": "Storage", "Lexar": "Storage", "QNAP": "Storage",
    # Displays
    "LG": "Displays & Monitors", "BenQ": "Displays & Monitors",
    # Cables & Adapters
    "Moshi": "Cables & Adapters", "Satechi": "Cables & Adapters", "Belkin": "Cables & Adapters",
    "Kensington": "Cables & Adapters", "NewerTech": "Cables & Adapters", "Startech": "Cables & Adapters",
    # Power & Charging
    "Mophie": "Power & Charging", "Allocacoc": "Power & Charging",
    # Professional Tools
    "Wacom": "Professional Tools", "Sonnet": "Professional Tools", "Synology": "Professional Tools",
    # Mobile & Accessories / Cases
    "Just Mobile": "Mobile & Accessories", "Elago": "Mobile & Accessories", "Twelve South": "Cases & Protection",
    "Muvit": "Cases & Protection", "Griffin": "Mobile & Accessories", "STM": "Cases & Protection",
    "Maclocks": "Cases & Protection", "Henge Docks": "Mobile & Accessories", "Spek SeeThru": "Cases & Protection",
    # Apple
    "Apple": "Mobile & Accessories",  # will usually be overridden by text matching for finer categorization
}

# Apply only to Other rows
mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()

# Text-first inference
text_inferred = product_category_df.loc[mask_other].apply(
    lambda r: infer_category_from_text_strong(r.get("name", ""), r.get("desc", "")),
    axis=1
)
product_category_df.loc[mask_other & text_inferred.notna(), "category"] = text_inferred[text_inferred.notna()]

# Brand fallback
mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
brand_inferred = product_category_df.loc[mask_other, "brand"].map(brand_defaults)
product_category_df.loc[mask_other & brand_inferred.notna(), "category"] = brand_inferred[brand_inferred.notna()]

# Final: Apple-specific refinement for any remaining Unknowns that are clearly in Apple ecosystem
mask_other = product_category_df["category"].eq("Other") | product_category_df["category"].isna()
def apple_hint(name, desc, brand):
    text = f"{str(name)} {str(desc)}".lower()
    if (str(brand).lower() == "apple" or
        re.search(r"\b(iphone|ipad|ipod|macbook|i\.?mac|mac\s?mini|mac\s?pro|apple\s?watch|airpods?|airtag|magsafe|lightning|thunderbolt)\b", text)):
        # If nothing else matched, assign a reasonable Apple bucket from text
        # Try the strong text rules again (they also catch Apple sub-families)
        cat = infer_category_from_text_strong(name, desc)
        return cat or "Mobile & Accessories"
    return None

apple_inferred = product_category_df.loc[mask_other].apply(
    lambda r: apple_hint(r.get("name",""), r.get("desc",""), r.get("brand","")),
    axis=1
)
product_category_df.loc[mask_other & apple_inferred.notna(), "category"] = apple_inferred[apple_inferred.notna()]

# Any residuals remain Other
product_category_df["category"] = product_category_df["category"].fillna("Other")

# Report
remaining_other = (product_category_df["category"] == "Other").sum()
print(f"Remaining Other after brand/name/desc rules: {remaining_other}")           


Remaining Other after brand/name/desc rules: 154


In [8]:
# Python
examples = product_category_df.loc[product_category_df["category"]=="Other", ["sku","name","desc","type"]].head(10)
examples

,sku,name,desc,type
122,FIT0009,Fitbit Aria scale smart white,smart scale with WiFi connection.,11905404
123,FIT0010,Fitbit Aria scale smart black,smart scale with WiFi connection.,11905404
445,FIT0019,Replacement Fitbit Flex 3 colors bracelet size L,Set of 3 colored bracelets Great for Fitbit Fl...,11905404
446,FIT0020,Replacement fitbit Flex 3 bracelets colors Size S,Set of 3 colored bracelets size small for Fitb...,11905404
454,KOU0007,Outdoor Koubachi Wi-Fi Plant Sensor,Wi-Fi sensor for remote control and plant care.,11905404
652,PAR0013,Parrot Flower Power Wireless Sensor Brown Plants,Wi-Fi sensor for remote control and plant care.,11905404
660,PAR0015,Parrot Flower Power Plants Wireless Sensor Green,Wi-Fi sensor for remote control and plant care.,11905404
852,NEA0010,Netatmo Pluviometer for weather station,Pluviometer add-on module for weather station.,11905404
1069,PAR0027,Parrot Central Cross Rolling Spider MiniDrone,Rolling MiniDrone Central Cross Spider.,11905404
1072,PAR0037,Upper body MiniDrone Parrot Jumping Sumo White,MiniDrone upper body Sumo Jumping.,11905404


In [9]:
print(product_category_df["category"].value_counts(dropna=False).head(10))
product_category_df
# (optional) Save or continue your pipeline
# product_category_df.to_csv("../data/categorized-data/product_category_df.csv", index=False)

category
Storage                 2758
Cases & Protection      2129
Computers & Laptops     1510
Mobile & Accessories    1232
Cables & Adapters        793
Audio & Input            521
Power & Charging         343
Displays & Monitors      253
Other                    154
Services                 153
Name: count, dtype: int64


,sku,name,desc,price,in_stock,type,category,brand
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,1,8696,Mobile & Accessories,Rain Design
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,0,13855401,Audio & Input,Apple
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,0,1387,Audio & Input,Apple
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,0,1230,Cables & Adapters,Apple
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,1,1364,Storage,Kingston
...,...,...,...,...,...,...,...,...
9987,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,1,12282,Mobile & Accessories,Belkin
9988,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,1,1392,Cases & Protection,Thule
9989,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,1,1392,Cases & Protection,Thule
9990,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,0,1392,Cases & Protection,Thule


In [10]:
# Python
product_category_df.loc[product_category_df["category"]=="Other", ["sku","brand","name","desc","price"]].head(20)

,sku,brand,name,desc,price
122,FIT0009,Fitbit,Fitbit Aria scale smart white,smart scale with WiFi connection.,119.99
123,FIT0010,Fitbit,Fitbit Aria scale smart black,smart scale with WiFi connection.,119.99
445,FIT0019,Fitbit,Replacement Fitbit Flex 3 colors bracelet size L,Set of 3 colored bracelets Great for Fitbit Fl...,25.99
446,FIT0020,Fitbit,Replacement fitbit Flex 3 bracelets colors Size S,Set of 3 colored bracelets size small for Fitb...,25.99
454,KOU0007,Other/Other,Outdoor Koubachi Wi-Fi Plant Sensor,Wi-Fi sensor for remote control and plant care.,119.00
652,PAR0013,Parrot,Parrot Flower Power Wireless Sensor Brown Plants,Wi-Fi sensor for remote control and plant care.,59.95
660,PAR0015,Parrot,Parrot Flower Power Plants Wireless Sensor Green,Wi-Fi sensor for remote control and plant care.,59.95
852,NEA0010,Netatmo,Netatmo Pluviometer for weather station,Pluviometer add-on module for weather station.,69.99
1069,PAR0027,Parrot,Parrot Central Cross Rolling Spider MiniDrone,Rolling MiniDrone Central Cross Spider.,5.99
1072,PAR0037,Parrot,Upper body MiniDrone Parrot Jumping Sumo White,MiniDrone upper body Sumo Jumping.,2.99


## Adding brand for categorized data frame

In [11]:
brands = pd.read_csv("../data/raw-data/brands.csv", dtype=str)

In [12]:

brands.columns = brands.columns.str.strip().str.lower()  # -> ['short','long']

brands["short"] = brands["short"].str.strip().str.upper()
brands["long"] = brands["long"].str.strip()

product_category_df["sku"] = (
    product_category_df["sku"].astype(str).str.strip().str.upper()
)

product_category_df["brand_code"] = product_category_df["sku"].str[:3]

code_to_brand = dict(zip(brands["short"], brands["long"]))
product_category_df["brand"] = product_category_df["brand_code"].map(code_to_brand)

product_category_df["brand"] = product_category_df["brand"].fillna("Unknown/Other")
product_category_df = product_category_df.drop(columns=["brand_code"])

product_category_df.groupby('category').count().sort_values(by='sku', ascending=False)

,sku,name,desc,price,in_stock,type,brand
category,,,,,,,
Storage,2758,2758,2758,2758,2758,2758,2758
Cases & Protection,2129,2129,2129,2129,2129,2129,2129
Computers & Laptops,1510,1510,1510,1510,1510,1510,1510
Mobile & Accessories,1232,1232,1232,1232,1232,1232,1232
Cables & Adapters,793,793,793,793,793,793,793
Audio & Input,521,521,521,521,521,521,521
Power & Charging,343,343,343,343,343,343,343
Displays & Monitors,253,253,253,253,253,253,253
Other,154,154,154,154,154,154,154


## Finally, save the file

In [13]:
product_category_df.to_csv("../data/categorized-data/product_category_df.csv")